In [270]:
import os
import numpy as np
import pandas as pd
import tempfile

import pandapower as pp
import pandapower.networks as nw
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import pandapower.plotting as pplt
import matplotlib.pyplot as plt
from pandapower.plotting.plotly import pf_res_plotly

#creating our net
net = nw.create_cigre_network_mv()

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')

#print(net)

#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, color ="g", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()

#Adding CS at rated power (6.6 kW)


max_line_loading=100

max_vm_pu= 1.1

min_vm_pu= 0.9

net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True


In [271]:

#importing the active and reactive power of loads from excel
df_active = pd.read_csv('active_power_time_series_ev_load.csv')
df_reactive = pd.read_csv('reactive_power_time_series_ev_load.csv')
# create the data source from
ds_active = DFData(df_active)
ds_reactive = DFData(df_reactive)

def timeseries_example(output_dir):
    
    #defing the time step
    n_timesteps = 24
    
    # 3. create controllers (to control P values of the load) ds_p is for active power and ds_q for reactive power
    create_controllers(net, ds_active, ds_reactive)

    # time steps to be calculated. Could also be a list with non-consecutive time steps
    time_steps = range(0, n_timesteps)

    # 4. the output writer with the desired results to be stored to files.
    ow = create_output_writer(net, time_steps, output_dir=output_dir)

    # 5. the main time series function
    run_timeseries(net, time_steps) 
    
    
load_index=net.load.index
load_name=net.load.name
def create_controllers(net,ds_active,ds_reactive):
    for i in range(len(load_index)):
        ConstControl(net, element='load', variable='p_mw', element_index=load_index[i],
                 data_source=ds_active, profile_name=load_name[i])
        ConstControl(net, element='load', variable='q_mvar', element_index=load_index[i],
                 data_source=ds_reactive, profile_name=load_name[i])

def create_output_writer(net, time_steps, output_dir):
    ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())
    # these variables are saved to the harddisk after / during the time series loop
    ow.log_variable('res_load', 'p_mw')
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_line', 'loading_percent')
    ow.log_variable('res_line', 'i_ka')
    ow.log_variable('res_trafo','loading_percent')
    return ow


#

In [272]:
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 112.61it/s]


In [273]:
ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)
    
vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

if any(line_loading.max()>100):
        print("Overloaded lines")

elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")

In [274]:
#lowlimit=350
#uplimit=400

#Bus CS 14 (0.63 MVA) - trafo id: 2

#for i in range (1,41):
 #   l = str(i)
  #  pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 11 (0.4 MVA) - trafo id: 3
#for i in range (uplimit,uplimit+20):
#    l = str(i)
 #   pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 5 (0.4 MVA) - trafo id: 4
#for i in range (uplimit+20,uplimit+40):
  #  l = str(i)
  #  pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
#Bus CS 4 (0.25 MVA) - trafo id: 5
#for i in range (uplimit+40,uplimit+50):
 #   l = str(i)
 #   pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
#Bus CS 9 (0.25 MVA)- trafo id: 6
#for i in range (uplimit+50,uplimit+60):
  #  l = str(i)
 #   pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)




In [275]:

lowlimit1=91
uplimit1=120

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (1,lowlimit1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit1
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit1 and all(trafo_loading.max()<100):
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
    trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
    trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit1:
        print("no maximum has been found")
        
    elif any(trafo_loading.max()>100):
        print("overloaded trafo")
        
#creating our net
net = nw.create_cigre_network_mv()

net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)


trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

92
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 93.34it/s]


93
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 90.30it/s]


overloaded trafo
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 118.30it/s]


In [276]:
trafo_loading.max()

0    79.843121
1    81.995791
2    99.765200
3     0.341654
4     0.341638
5     0.323989
6     0.323927
dtype: float64

In [277]:
c15=c-1
EVs_B15=c15

lowlimit2=c15+55
uplimit2=c15+100

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (c15,lowlimit2):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit2
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit2 and all(trafo_loading.max()<100):
    
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
    
    trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
    trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)


    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit2:
        print("no maximum has been found")
   
    elif any(trafo_loading.max()>100):
        print("overloaded trafo")
        
c16=c-1
EVs_B16=c16-c15        
print(EVs_B16)        
        
#creating our net
net = nw.create_cigre_network_mv()
net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)


trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)


148
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 107.39it/s]


149
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 84.15it/s]


150
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 77.28it/s]


overloaded trafo
57
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 118.63it/s]


In [278]:
trafo_loading.max()

0    80.666267
1    82.833677
2    99.999361
3    99.571839
4     0.340790
5     0.323178
6     0.323141
dtype: float64

In [279]:


lowlimit3=c16+57
uplimit3=c16+70


for i in range (c16,lowlimit3):
    l = str(i)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit3
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit3  and all(trafo_loading.max()<100):
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
    trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
    trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit3:
        print("no maximum has been found")
    elif any(trafo_loading.max()>100):
        print("overloaded trafo")
        
        
c17=c-1
EVs_B17=c17-c16       
print(EVs_B17)        
        
#creating our net
net = nw.create_cigre_network_mv()
net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c16,c17):
    l = str(i)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)


trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

207
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 102.39it/s]


overloaded trafo
57
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 110.73it/s]


In [280]:
all(trafo_loading.max()<100)

True

In [281]:
print(trafo_loading.max())

0    81.489479
1    83.656802
2    99.114264
3    99.877721
4    99.920934
5     0.322362
6     0.322375
dtype: float64


In [282]:
trafo_loading

,0,1,2,3,4,5,6
0,27.621479,28.613024,95.076239,95.330442,95.345636,0.321728,0.321741
1,26.667978,27.638752,95.012245,95.258450,95.273206,0.321963,0.321976
2,25.683036,26.632379,94.946391,95.184365,95.198670,0.322206,0.322218
3,25.678092,26.627327,94.946061,95.183994,95.198296,0.322207,0.322220
4,25.045630,25.981119,94.903908,95.136573,95.150587,0.322362,0.322375
5,25.539249,26.485465,94.936798,95.173574,95.187813,0.322241,0.322254
6,26.728570,27.700664,95.016305,95.263017,95.277801,0.321948,0.321961
7,30.000767,31.044220,95.236972,95.511269,95.527563,0.321138,0.321153
8,41.362150,42.654185,96.025575,96.398565,96.420271,0.318278,0.318297
9,51.377696,52.888909,96.750758,97.214754,97.241459,0.315695,0.315717


In [283]:


lowlimit4=c17+10
uplimit4=c17+60

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (c17,lowlimit4):
    l = str(i)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
c=lowlimit4
while all(vm_pu.max()<max_vm_pu) and all(vm_pu.min()>min_vm_pu) and all(line_loading.max()<100) and c<uplimit4  and all(trafo_loading.max()<100):
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
    trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
    trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit4:
        print("no maximum has been found")
        
    elif any(trafo_loading.max()>100):
        print("overloaded trafo")
        
c18=c-1
EVs_B18=c18-c17        
print(EVs_B18)        
        
#creating our net
net = nw.create_cigre_network_mv()
net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c16,c17):
   l = str(i)
   pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c17,c18):
    l = str(i)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

217
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 96.23it/s]


218
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 88.51it/s]


219
Results can be found in your local temp folder: time_series_ev


...

100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 66.88it/s]


221
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 61.54it/s]


overloaded trafo
14
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 109.63it/s]


In [284]:
print(trafo_loading.max())

0    81.693201
1    83.857380
2    99.168627
3    99.950992
4    99.995045
5    39.023484
6     0.322193
dtype: float64


In [287]:


lowlimit5=c18+0
uplimit5=c18+40

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (c18,lowlimit5):
    l = str(i)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit5
while all(vm_pu.max()<max_vm_pu) and all(vm_pu.min()>min_vm_pu) and all(line_loading.max()<100) and c<uplimit5  and all(trafo_loading.max()<100):
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)
    
    trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
    trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit5:
        print("no maximum has been found")
        
    elif any(net.res_trafo.loading_percent>100):
        print("overloaded trafo")
        
    elif any(trafo_loading.max()>100):
        print("overloaded trafo")
        
c19=c-1
EVs_B19=c19-c18        
print(EVs_B19)        
        
#creating our net
net = nw.create_cigre_network_mv()
net.switch.closed[1] = True
net.switch.closed[2] = True
net.switch.closed[4] = True

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c16,c17):
   l = str(i)
   pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c17,c18):
    l = str(i)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
for i in range (c18,c19):
    l = str(i)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

trafo_loading_file = os.path.join(output_dir, "res_trafo", "loading_percent.xlsx")
trafo_loading = pd.read_excel(trafo_loading_file, index_col=0)

221
Results can be found in your local temp folder: time_series_ev


100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 96.11it/s]


overloaded trafo
0
Results can be found in your local temp folder: time_series_ev


100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 100.42it/s]


In [290]:
net.res_trafo.loading_percent

0    29.175903
1    30.195920
2    95.211946
3    95.493183
4    95.509681
5    37.348462
6     0.321220
Name: loading_percent, dtype: float64

In [291]:
print(trafo_loading.max())

0    81.693201
1    83.857380
2    99.168627
3    99.950992
4    99.995045
5    39.023484
6     0.322193
dtype: float64


In [211]:
all(vm_pu.max()<max_vm_pu) and all(vm_pu.min()>min_vm_pu) and all(line_loading.max()<100) and c<uplimit4  and all(trafo_loading.max()<100)

True